# Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import config
from joblib import dump, load
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from utils import confusion
from GMHI import GMHI

# Load data

In [2]:
X = pd.read_csv(os.path.join(config.CLEAN_DIR, "taxonomic_features.csv"), index_col=[0, 1])
print(X.shape)
y = pd.read_csv(os.path.join(config.CLEAN_DIR, "metadata.csv"), index_col=[0, 1]).iloc[:, [0]]
display(X.head())
display(y.head())

(8069, 3200)


k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID Sample Accession                                            
GMHI-10  SAMN03283239        0.000000                     0.000000   
         SAMN03283266        0.000000                     0.000000   
         SAMN03283281        0.009764                     0.009764   
         SAMN03283294        0.000000                     0.000000   
         SAMN03283288        0.011865                     0.011865   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID Sample Accession                                                   
GMHI-10  SAMN03283239                                            0.000000   
         SAMN03283266                                            0.000000   
         SAMN03283281                                            0.009764   
         SAMN03283294                                            0.000000   
         SAMN03283288                                            0.011865   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID Sample Accession                                                                         
GMHI-10  SAMN03283239                                               0.000000                      
         SAMN03283266                                               0.000000                      
         SAMN03283281                                               0.009764                      
         SAMN03283294                                               0.000000                      
         SAMN03283288                                               0.011865                      

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae  \
Study_ID Sample Accession                                                                                                
GMHI-10  SAMN03283239                                               0.000000                                             
         SAMN03283266                                               0.000000                                             
         SAMN03283281                                               0.009764                                             
         SAMN03283294                                               0.000000                                             
         SAMN03283288                                               0.011865                                             

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter  \
Study_ID Sample Accession                                                                                                                      
GMHI-10  SAMN03283239                                               0.000000                                                                   
         SAMN03283266                                               0.000000                                                                   
         SAMN03283281                                               0.009764                                                                   
         SAMN03283294                                               0.000000                                                                   
         SAMN03283288                                               0.011865                                                                   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
GMHI-10  SAMN03283239                                               0.000000              

PHENOTYPE:Healthy_Nonhealthy
Study_ID Sample Accession                              
GMHI-10  SAMN03283239                              True
         SAMN03283266                              True
         SAMN03283281                              True
         SAMN03283294                              True
         SAMN03283288                              True

# Train Models!

## Train GMHI model on entire dataset

In [3]:
# use same parameters as original paper
gmhi = GMHI(use_shannon=True, theta_f=1.4, theta_d=0.1)

# use only non-viral species
X_GMHI = X.copy()
X_GMHI = X_GMHI[list(filter(lambda x : "s__" in x and "virus" not in x, X_GMHI.columns))]

gmhi.fit(X_GMHI, y)
GMHI_scores = pd.DataFrame(gmhi.decision_function(X_GMHI), index=y.index, columns=["GMHI"])
display(GMHI_scores)
print("GMHI balanced_accuracy on training set:", balanced_accuracy_score(y, GMHI_scores > 0))
idx = (abs(GMHI_scores) >= 0.1).values
print("GMHI balanced_accuracy on training set for |GMHI| >= 0.1:", balanced_accuracy_score(y[idx], GMHI_scores[idx] > 0))
idx = (abs(GMHI_scores) >= 1).values
print("GMHI balanced_accuracy on training set for |GMHI| >= 1:", balanced_accuracy_score(y[idx], GMHI_scores[idx] > 0))

GMHI
Study_ID  Sample Accession          
GMHI-10   SAMN03283239      3.832094
          SAMN03283266      1.758548
          SAMN03283281      3.341858
          SAMN03283294      1.429055
          SAMN03283288      1.064592
...                              ...
Yang_2020 SRR6456373       -4.472053
          SRR6456374       -2.158801
          SRR6456375        0.280615
          SRR6456376        1.132266
          SRR6456377       -0.305132

[8069 rows x 1 columns]

GMHI balanced_accuracy on training set: 0.7172501242714733
GMHI balanced_accuracy on training set for |GMHI| >= 0.1: 0.7228776593558975
GMHI balanced_accuracy on training set for |GMHI| >= 1: 0.785641042415692


## Train GMHI2 model on entire dataset

In [4]:
gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=config.REGULARIZATION, class_weight="balanced")

X_GMHI2 = X > config.PRESENCE_CUTOFF
X_GMHI2

k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID  Sample Accession                                            
GMHI-10   SAMN03283239           False                        False   
          SAMN03283266           False                        False   
          SAMN03283281            True                         True   
          SAMN03283294           False                        False   
          SAMN03283288            True                         True   
...                                ...                          ...   
Yang_2020 SRR6456373             False                        False   
          SRR6456374             False                        False   
          SRR6456375             False                        False   
          SRR6456376              True                         True   
          SRR6456377             False                        False   

                            k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID  Sample Accession                                                   
GMHI-10   SAMN03283239                                               False   
          SAMN03283266                                               False   
          SAMN03283281                                                True   
          SAMN03283294                                               False   
          SAMN03283288                                                True   
...                                                                    ...   
Yang_2020 SRR6456373                                                 False   
          SRR6456374                                                 False   
          SRR6456375                                                 False   
          SRR6456376                                                  True   
          SRR6456377                                                 False   

                            k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID  Sample Accession                                                                         
GMHI-10   SAMN03283239                                                  False                      
          SAMN03283266                                                  False                      
          SAMN03283281                                                   True                      
          SAMN03283294                                                  False                      
          SAMN03283288                                                   True                      
...                                                                       ...                      
Yang_2020 SRR6456373                                                    False                      
          SRR6456374                                                    False                      
          SRR6456375                                                    False                      
          SRR6456376                                                     True                      
          SRR6456377                                                    False                      

                            k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae  \
Study_ID  Sample Accession                                                                                                
GMHI-10   SAMN03283239                                                  False                                             
          SAMN03283266                                                  False                                             
          SAMN03283281                                                   True                                             
          SAMN03283294                                                  False                                             
          SAMN03283288                            

In [5]:
gmhi2.fit(X_GMHI2, y.values.flatten())
GMHI2_scores = pd.DataFrame(gmhi2.decision_function(X_GMHI2), index=y.index, columns=["GMHI2"])
display(GMHI2_scores)
print("GMHI2 balanced_accuracy on training set:", balanced_accuracy_score(y, GMHI2_scores > 0))
idx = (abs(GMHI2_scores) >= 0.5).values
print("GMHI2 balanced_accuracy on training set |GMHI2| >= 0.5:", balanced_accuracy_score(y[idx], GMHI2_scores[idx] > 0))
idx = (abs(GMHI2_scores) >= 1).values
print("GMHI2 balanced_accuracy on training set |GMHI2| >= 1:", balanced_accuracy_score(y[idx], GMHI2_scores[idx] > 0))

GMHI2
Study_ID  Sample Accession          
GMHI-10   SAMN03283239      1.835237
          SAMN03283266      0.103722
          SAMN03283281      1.116329
          SAMN03283294      1.758818
          SAMN03283288      1.675791
...                              ...
Yang_2020 SRR6456373       -1.029233
          SRR6456374       -0.941967
          SRR6456375       -0.030621
          SRR6456376        0.973798
          SRR6456377       -0.066339

[8069 rows x 1 columns]

GMHI2 balanced_accuracy on training set: 0.7988526637127441
GMHI2 balanced_accuracy on training set |GMHI2| >= 0.5: 0.8575150415882389
GMHI2 balanced_accuracy on training set |GMHI2| >= 1: 0.9098325472786429


In [6]:
display(confusion(GMHI2_scores, y, 0))
display(confusion(GMHI2_scores, y, 0.1))
display(confusion(GMHI2_scores, y, 0.5))
display(confusion(GMHI2_scores, y, 1))

,Predicted Healthy,Predicted Nonhealthy,Accuracy
Actual Nonhealthy,489.0,2033.0,0.806106
Actual healthy,4391.0,1156.0,0.791599


,Predicted Healthy,Predicted Nonhealthy,Accuracy
Actual Nonhealthy,434.0,1976.0,0.819917
Actual healthy,4268.0,1041.0,0.803918


,Predicted Healthy,Predicted Nonhealthy,Accuracy
Actual Nonhealthy,264.0,1678.0,0.864058
Actual healthy,3763.0,659.0,0.850972


,Predicted Healthy,Predicted Nonhealthy,Accuracy
Actual Nonhealthy,115.0,1263.0,0.916546
Actual healthy,3011.0,323.0,0.903119


# Export stuff

## Export GMHI scores and serialize model

In [7]:
GMHI_scores.to_csv(os.path.join(config.LOG_DIR, "GMHI_scores_training_set.csv"))
dump(gmhi, os.path.join(config.MODEL_DIR, 'gmhi_model.joblib'))

['../output/models/gmhi_model.joblib']

## Export GMHI2 scores and serialize model

In [8]:
GMHI2_scores.to_csv(os.path.join(config.LOG_DIR, "GMHI2_scores_training_set.csv"))
dump(gmhi2, os.path.join(config.MODEL_DIR, 'gmhi2_model.joblib'))

['../output/models/gmhi2_model.joblib']